<a href="https://colab.research.google.com/github/trivendrakulhare/TensorFlow_MNIST/blob/master/MNIST_Digit_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Deep Neural Network for MNIST Classification
# The dataset provides 70,000 images (28x28 pixels) of handwritten digits (1 digit per image). 60_000 Training data and 10_000 testing data
# The goal is to write an algorithm that detects which digit is written. Since there are only 10 digits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), this is a classification problem with 10 classes.
#Our goal would be to build a neural network with 2 hidden layers.

In [0]:
## Import the relevant packages

import numpy as np
import tensorflow as tf

# TensorFLow includes a data provider for MNIST that we'll use.
import tensorflow_datasets as tfds

In [5]:
# mnist_dataset = tfds.load() loads the data.
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
# with_info=True will also provide us with a tuple containing information about the version, features, number of samples
# as_supervised=True will load the dataset in a 2-tuple structure (input, target) 

# extracting the training and testing dataset
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

#Here we split the validation dataset from training dataset.
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
#casting this number to an integer, as a float may cause an error along the way
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

# store the number of test samples in a dedicated variable 
num_test_samples = mnist_info.splits['test'].num_examples

num_test_samples = tf.cast(num_test_samples, tf.int64)

# Defining a function called: scale, that will take an MNIST image and its label
def scale(image, label):
   
    image = tf.cast(image, tf.float32)
    # since the possible values for the inputs are 0 to 255 (256 different shades of grey)
    # if we divide each element by 255, we would get the desired result -> all elements will be between 0 and 1 
    image /= 255.

    return image, label


# the method .map() allows us to apply a custom transformation to a given dataset
# we have already decided that we will get the validation data from mnist_train, so 
scaled_train_and_validation_data = mnist_train.map(scale)

# finally, we scale and batch the test data
# we scale it so it has the same magnitude as the train and validation
# there is no need to shuffle it, because we won't be training on the test data
# there would be a single batch, equal to the size of the test data
test_data = mnist_test.map(scale)


# let's also shuffle the data

BUFFER_SIZE = 1000
# this BUFFER_SIZE parameter is here for cases when we're dealing with enormous datasets
# then we can't shuffle the whole dataset in one go because we can't fit it all in memory
# so instead TF only stores BUFFER_SIZE samples in memory at a time and shuffles them

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)


validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

# similarly, the train_data is everything else, so we skip as many samples as there are in the validation dataset
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

# determine the batch size
BATCH_SIZE = 50

# batching train data would help us to iterate over the different batches
train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)

# batch the test data
test_data = test_data.batch(num_test_samples)


# takes next batch (it is the only batch)
# because as_supervized=True, we've got a 2-tuple structure
validation_inputs, validation_targets = next(iter(validation_data))

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.0. Subsequent calls will reuse this data.


In [0]:
# Outline the model

input_size = 784
output_size = 10
hidden_layer_size = 50
    
# define how the model will look like
model = tf.keras.Sequential([
    
    # the first layer (the input layer)
    # each observation is 28x28x1 pixels, therefore it is a tensor of rank 3
    # since we don't know CNNs yet, we don't know how to feed such input into our net, so we must flatten the images
  
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # input layer
    
    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation function
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    
    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

In [0]:
# we define the optimizer we'd like to use, 
# the loss function, 
# and the metrics we are interested in obtaining at each iteration
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [9]:
# Determine the maximum number of epochs
NUM_EPOCHS = 5

# we fit the model, specifying the
# training data
# the total number of epochs
# and the validation data we just created ourselves in the format: (inputs,targets)
STEPS = num_validation_samples/ BATCH_SIZE
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), validation_steps = STEPS, verbose =2)#

Epoch 1/5
1080/1080 - 14s - loss: 0.3526 - accuracy: 0.8964 - val_loss: 0.1894 - val_accuracy: 0.9432
Epoch 2/5
1080/1080 - 15s - loss: 0.1613 - accuracy: 0.9520 - val_loss: 0.1496 - val_accuracy: 0.9560
Epoch 3/5
1080/1080 - 15s - loss: 0.1233 - accuracy: 0.9637 - val_loss: 0.1440 - val_accuracy: 0.9580
Epoch 4/5
1080/1080 - 15s - loss: 0.1003 - accuracy: 0.9703 - val_loss: 0.1339 - val_accuracy: 0.9607
Epoch 5/5
1080/1080 - 14s - loss: 0.0846 - accuracy: 0.9742 - val_loss: 0.1264 - val_accuracy: 0.9622


In [10]:
# Test the model
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 1ms/step - loss: 0.1180 - accuracy: 0.9640


In [11]:
# Printing test loss and test accuracy
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.12. Test accuracy: 96.40%
